In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

┌ Info: Precompiling MixedModels [ff71e718-51f3-5ec2-a782-8ffcbfa3c316]
└ @ Base loading.jl:1317


#### using HighDimMixedModels

In [3]:
using HighDimMixedModels

┌ Info: Precompiling HighDimMixedModels [ced58d7a-d58c-464d-86d9-3a8462bebd28]
└ @ Base loading.jl:1317


**Set up data and formula, specify factor variables(CategoricalTerm)**

In [4]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


In [5]:
f = @formula(period ~ herd + incid + hsz)
contrasts = Dict( :incid => ContinuousTerm, :hsz => ContinuousTerm, :herd => CategoricalTerm)
#fit(MixedModel, @formula(period ~ incid + hsz +(1|herd)), df, REML = true)

Dict{Symbol, UnionAll} with 3 entries:
  :hsz   => ContinuousTerm
  :herd  => CategoricalTerm
  :incid => ContinuousTerm

In [6]:
f

FormulaTerm
Response:
  period(unknown)
Predictors:
  herd(unknown)
  incid(unknown)
  hsz(unknown)

**try for apply_schema**

In [26]:
LinearMixedModel<:MixedModel

true

In [27]:
highDimMixedModel<:MixedModel

true

In [28]:
# try for f2
f2 = @formula(period ~ incid + hsz +(1|herd))

FormulaTerm
Response:
  period(unknown)
Predictors:
  incid(unknown)
  hsz(unknown)
  (herd)->1 | herd

In [29]:
sch = schema(f2,df,contrasts)

StatsModels.Schema with 4 entries:
  period => period
  hsz => hsz
  incid => incid
  herd => herd


In [30]:
form = apply_schema(f2, sch)

FormulaTerm
Response:
  period(continuous)
Predictors:
  incid(continuous)
  hsz(continuous)
  (herd)->1 | herd

In [63]:
form = MixedModels.apply_schema(f2, sch, LinearMixedModel)  ## do not need MixedModels prefix due to multiple dispatch

FormulaTerm
Response:
  period(continuous)
Predictors:
  0
  incid(continuous)
  hsz(continuous)
  (1 | herd)

In [64]:
form = MixedModels.apply_schema(f2, sch, highDimMixedModel)

FormulaTerm
Response:
  period(continuous)
Predictors:
  0
  incid(continuous)
  hsz(continuous)
  (1 | herd)

#### 3.variable names indicates M,X,Z

In [12]:
HMM = highDimMixedModel(f, df, contrasts, "incid", "hsz", "herd")

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


In [13]:
ml = fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

LoadError: UndefVarError: fit not defined

In [14]:
ml = MixedModels.fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/zyxu/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Minimizing 16 	 Time: 0:00:00 (27.38 ms/it)


|             |    Est. |     SE |     z |      p | σ_herd |
|:----------- | -------:| ------:| -----:| ------:| ------:|
| incid       | -0.2939 | 0.0890 | -3.30 | 0.0010 |        |
| hsz         |  0.1317 | 0.0239 |  5.52 | <1e-07 |        |
| (Intercept) |         |        |       |        | 1.1569 |
| Residual    |  1.3496 |        |       |        |        |


In [15]:
sigma, betaM, betaX, opt = HighDimMixedModels.fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [1.0, 1.0]
negLog = -61.76979106719719
The initial object value is -61.76979106719719
sigma = [1.0, 1.0]
negLog = -61.76979106719719
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = -61.76979106719719
sigma = [1.75, 1.0]
negLog = -61.75310918184841
sigma = [1.0, 1.75]
negLog = -54.23568245082083
sigma = [0.25, 1.0]
negLog = -66.88313923192547
sigma = [1.0, 0.25]
negLog = -163.59464997549028
sigma = [0.29289321881345254, 0.0]
negLog = Inf
sigma = [0.9469669914110089, 0.19696699141100893]
negLog = -205.06915257224978
sigma = [0.995676667220638, 0.13993745974082217]
negLog = -285.5462857169225
sigma = [0.9952996645647639, 0.06493840728682465]
negLog = -616.906085145067
sigma = [0.9878315127277438, 0.06424796674686986]
negLog = -623.7150350631232
sigma = [0.987341243774038, 0.05676400814509819]
negLog = -707.1217774973741
sigma = [0.9948025184365511, 0.05600283772361585]
negLog = -716.767890276837
sigma = [0.9947292842170775, 0.05525642178789495]
negLog = -726.588670390

In [16]:
HMM::highDimMixedModel

Linear mixed model fit by REML
 period ~ herd + incid + hsz
 REML criterion at convergence: -743.9667300124138


σ_z : 0.9946902649028658
σ : 0.05398364462117065
β_M : [-0.2510119577349666]
β_X : Float64[]


In [17]:
HMM::MixedModel

Linear mixed model fit by REML
 period ~ herd + incid + hsz
 REML criterion at convergence: -743.9667300124138


σ_z : 0.9946902649028658
σ : 0.05398364462117065
β_M : [-0.2510119577349666]
β_X : Float64[]


In [18]:
HMM::StatsModels.RegressionModel

Linear mixed model fit by REML
 period ~ herd + incid + hsz
 REML criterion at convergence: -743.9667300124138


σ_z : 0.9946902649028658
σ : 0.05398364462117065
β_M : [-0.2510119577349666]
β_X : Float64[]


In [19]:
ml::LinearMixedModel

|             |    Est. |     SE |     z |      p | σ_herd |
|:----------- | -------:| ------:| -----:| ------:| ------:|
| incid       | -0.2939 | 0.0890 | -3.30 | 0.0010 |        |
| hsz         |  0.1317 | 0.0239 |  5.52 | <1e-07 |        |
| (Intercept) |         |        |       |        | 1.1569 |
| Residual    |  1.3496 |        |       |        |        |


In [20]:
ml::MixedModel

|             |    Est. |     SE |     z |      p | σ_herd |
|:----------- | -------:| ------:| -----:| ------:| ------:|
| incid       | -0.2939 | 0.0890 | -3.30 | 0.0010 |        |
| hsz         |  0.1317 | 0.0239 |  5.52 | <1e-07 |        |
| (Intercept) |         |        |       |        | 1.1569 |
| Residual    |  1.3496 |        |       |        |        |


In [21]:
ml::StatsModels.RegressionModel

|             |    Est. |     SE |     z |      p | σ_herd |
|:----------- | -------:| ------:| -----:| ------:| ------:|
| incid       | -0.2939 | 0.0890 | -3.30 | 0.0010 |        |
| hsz         |  0.1317 | 0.0239 |  5.52 | <1e-07 |        |
| (Intercept) |         |        |       |        | 1.1569 |
| Residual    |  1.3496 |        |       |        |        |


In [22]:
HMM::MixedModel

Linear mixed model fit by REML
 period ~ herd + incid + hsz
 REML criterion at convergence: -743.9667300124138


σ_z : 0.9946902649028658
σ : 0.05398364462117065
β_M : [-0.2510119577349666]
β_X : Float64[]


In [23]:
HMM::StatsModels.RegressionModel

Linear mixed model fit by REML
 period ~ herd + incid + hsz
 REML criterion at convergence: -743.9667300124138


σ_z : 0.9946902649028658
σ : 0.05398364462117065
β_M : [-0.2510119577349666]
β_X : Float64[]


In [ ]:
highDimMixedModel::MixedModel

In [40]:
HMM

Linear mixed model fit by REML
 period ~ herd + incid + hsz
 REML criterion at convergence: -743.9667300124138


σ_z : 0.9946902649028658
σ : 0.05398364462117065
β_M : [-0.2510119577349666]
β_X : Float64[]


In [39]:
HMM.formula

FormulaTerm
Response:
  period(continuous)
Predictors:
  herd(DummyCoding:15→14)
  incid(continuous)
  hsz(continuous)

In [41]:
first(HMM.y,5)

5-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0
 1.0

In [42]:
first(HMM.M.M,5)

5-element Vector{Float64}:
 2.0
 3.0
 4.0
 0.0
 3.0

In [43]:
first(HMM.X.X,5)

5-element Vector{Float64}:
 14.0
 12.0
  9.0
  5.0
 22.0

In [54]:
HMM.Z.Z ## This is wrong! Should have 15 columns

56×14 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮              


### Now try to use apply_schema in MixedModels

In [58]:
# try for f2
f2 = @formula(period ~ incid + hsz +(1|herd))
sch = schema(f2,df)
form = apply_schema(f2, sch, highDimMixedModel) ## multiple dispatch

FormulaTerm
Response:
  period(continuous)
Predictors:
  0
  incid(continuous)
  hsz(continuous)
  (1 | herd)

In [59]:
y,X = modelcols(form,df)

([1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 1.0, 2.0, 3.0  …  3.0, 4.0, 1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 4.0], ([2.0 14.0; 3.0 12.0; … ; 1.0 15.0; 0.0 15.0], [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 1.0]))

In [60]:
Z = X[2]

56×15 ReMat{Float64, 1}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [61]:
rank(Z)

15

In [62]:
df.herd

56-element Arrow.DictEncoded{String, Int8, Arrow.List{String, Int32, Vector{UInt8}}}:
 "H01"
 "H01"
 "H01"
 "H01"
 "H02"
 "H02"
 "H02"
 "H03"
 "H03"
 "H03"
 "H03"
 "H04"
 "H04"
 ⋮
 "H13"
 "H13"
 "H13"
 "H13"
 "H14"
 "H14"
 "H14"
 "H14"
 "H15"
 "H15"
 "H15"
 "H15"